In [19]:
import ads
import pandas as pd
ads.config.token='42guhMWZUbJFFB5XPzhLJ14LScoO240jZvlkzPk3'

In [24]:
papers = ads.SearchQuery(q='(((full:"tardis" AND (full:"kerzendorf" OR (bibstem:"Natur" AND full:"supernova")))) AND year:2014-)+property:refereed', sort="date")

bibcodes = []
titles = []
first_authors = []

for paper in papers:
    bibcodes.append(paper.bibcode)
    titles.append(paper.title)
    first_authors.append(paper.first_author)

In [25]:
d = {'bibcode': bibcodes, 'title': titles, 'first author': first_authors}
df = pd.DataFrame(data=d)
df.to_csv('adslist.csv')